<a href="https://colab.research.google.com/github/assafrbi/mini-project-Digital-Humanities/blob/main/dh_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from requests import get
from bs4 import BeautifulSoup
import re
from time import sleep
from random import randint
from warnings import warn
from tqdm import tqdm
from imdb import Cinemagoer
import pandas as pd

### Load Data

In [2]:
url = "https://www.imdb.com/search/title/?title_type=feature&countries=il&start=1&ref_=adv_nxt"
response = get(url)
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

num_films_text = html_soup.find_all('div', class_ = 'desc')
num_films=re.search('of (\d.+) titles',str(num_films_text[0])).group(1)
num_films=int(num_films.replace(',', ''))
print(num_films)

ids = []
requests = 0

# For every page in the interval`
for start in tqdm(range(1, 2, 50)):    
    # Make a get request    
    url = "https://www.imdb.com/search/title/?title_type=feature&countries=il&start="+str(start)+"&ref_=adv_nxt"
    response = get(url)

    # Pause the loop
    sleep(randint(8,15))  

    # Monitor the requests
    requests += 1
    sleep(randint(1,3))

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))   

    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')

    # Select all the 50 movie containers from a single page
    movie_containers = page_html.find_all('div', class_ = 'lister-item mode-advanced')

    # Scrape the ID 
    for i in range(len(movie_containers)):
        id = re.search('tt(\d+)/',str(movie_containers[i].a)).group(1)
        ids.append(id)
print(ids)

1471


100%|██████████| 1/1 [00:09<00:00,  9.90s/it]

['9100054', '3605418', '0090859', '3454424', '0090927', '1226753', '5448338', '6952960', '15041866', '0092534', '9733388', '1821641', '8000718', '0081459', '1546421', '5968274', '4552524', '0087682', '2318625', '0091019', '4191702', '0095382', '14043818', '1135989', '7942736', '6333086', '4777584', '0079118', '2924590', '6896536', '0201485', '0782867', '2309224', '0214871', '12494638', '0334754', '0479341', '2926810', '1424327', '14556392', '0095182', '4830786', '1032856', '5022872', '7016254', '0069786', '0108350', '0352994', '1674773', '0476643']


In [13]:
ia = Cinemagoer()
titles = []
synopses = []

for id in tqdm(ids):
  movie = ia.get_movie(id)
  title = movie.get('title')
  synopsis = movie.get('plot')
  more = movie.get_current_info()
  titles.append(title)
  synopses.append(synopsis)

  4%|▍         | 2/50 [00:05<02:02,  2.55s/it]


KeyboardInterrupt: ignored

In [4]:
films = { 'title': titles, 'synopsis': synopses}
df = pd.DataFrame(films)
df

,title,synopsis
0,The Lost Daughter,[A woman's beach vacation takes a dark turn wh...
1,Knock Knock,[A devoted father helps two stranded young wom...
2,Cobra,[A tough-on-crime street cop must protect the ...
3,Where Is Anne Frank,"[The film follows the journey of Kitty, the im..."
4,The Delta Force,"[After a plane is hijacked by terrorists, The ..."
5,The Debt,"[In 1965, three Mossad Agents cross into East ..."
6,Plan A,"[In 1945, a group of Jewish holocaust survivor..."
7,The Kindergarten Teacher,[A kindergarten teacher in New York becomes ob...
8,The Swimmer,[Sees the discriminative tendencies in sports ...
9,Allan Quatermain and the Lost City of Gold,[Allan Quatermain once again teams up with Jes...


In [8]:
from google.colab import drive
drive.mount('/content/gdrive')
with open('/content/gdrive/My Drive/file.txt', 'w') as f:
  f.write(df.to_csv())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Text Preprocessing


### Topic Modeling - Latent Dirichlet Allocation